In [1]:
import numpy as np
import awkward as ak
from coffea import processor, util

%matplotlib inline
import matplotlib.pyplot as plt

import boostedhiggs

In [2]:
class HbbProcessor(processor.ProcessorABC):
    def __init__(self, year='2017'):
        self._year = year
        
        self._btagWPs = {
            'med': {
                '2016': 0.6321,
                '2017': 0.4941,
                '2018': 0.4184,
            },
        }
        
        self._accumulator = processor.dict_accumulator({
            "cutflow": processor.defaultdict_accumulator(float),
        })

    @property
    def accumulator(self):
        return self._accumulator

    def process(self, df):
        events = boostedhiggs.buildevents(df)
        output = self.accumulator.identity()

        fatjets = events.fatjets
        fatjets['msdcorr'] = boostedhiggs.corrected_msoftdrop(fatjets)
        fatjets['rho'] = 2*np.log(fatjets.msdcorr / fatjets.p4.pt)
        fatjets['n2ddt'] = fatjets.n2 - boostedhiggs.n2ddt_shift(fatjets, year=self._year)
        
        leadingjet = fatjets[:, 0:1]
        good = ((leadingjet.p4.pt > 450)
                & (leadingjet.p4.eta < 2.4)
                & (leadingjet.jetId & 2).astype(bool)  # tight
                & (leadingjet.n2ddt < 0.)
                & (leadingjet.msdcorr > 40)
               )
        good_event = good.all()
        jets = events.jets
        
        ak4_ak8_pair = jets.cross(leadingjet, nested=True)
        dphi = ak4_ak8_pair.i0.p4.delta_phi(ak4_ak8_pair.i1.p4)
        opp = (np.abs(dphi) > np.pi / 2).all()
        # only consider first 4 jets to be consistent with old framework
        ak4_opposite = jets[opp & (jets.p4.pt > 30.) & (jets.localindex < 4)]
        good_event &= ak4_opposite.deepcsvb.max() < self._btagWPs['med'][self._year]
        
        nelectrons = ((events.electrons.cutBased & (1<<2)).astype(bool) & (events.electrons.p4.pt > 20)).counts
        nmuons = (events.muons.looseId & (events.muons.p4.pt > 20)).counts
        ntaus = (events.taus.idDecayMode & (events.taus.p4.pt > 20)).counts
        good_event &= (events.met.rho < 140.) & (nmuons == 0) & (nelectrons == 0) & (ntaus == 0)
        
        output['cutflow']['pass all'] += good_event.sum()
        
        return output

    def postprocess(self, accumulator):
        return accumulator

In [3]:
files = {
    'ttbar': [
        'data/TTToHadronic_TuneCP5_13TeV-powheg-pythia8_10X.root'
    ]
}

p = HbbProcessor(year='2018')
exe_config = {
    'flatten': True,
    'workers': 4,
    'savemetrics': True,
}
output, metrics = processor.run_uproot_job(files, 'Events', p, processor.futures_executor, exe_config)

In [4]:
evtrate = metrics['entries'].value / metrics['processtime'].value
print("Events / s / thread: %.0f" % evtrate)

Events / s / thread: 51060
